In [2]:
input_string = "abacdbee"
output_string = list(set(input_string))

In [3]:
# set implementation in python
def simple_set_dedup(input_string):
  s = set()
  r = []
  for c in input_string:
    if c not in s:
      s.add(c)
      r.append(c)

  return r

In [4]:
simple_set_dedup(input_string)

['a', 'b', 'c', 'd', 'e']

In [5]:
import numpy as np
from numpy.typing import NDArray

In [6]:
default_tokens = {c: x for x, c in enumerate('abcde')}

def tokenize(input_string, token_mapping=None):
  if token_mapping is None:
    token_mapping = default_tokens
  return np.array([token_mapping[c] for c in input_string]), len(token_mapping)

In [7]:
default_tokens

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}

In [8]:
tokens, alphabet_size = tokenize(input_string)
tokens, alphabet_size

(array([0, 1, 0, 2, 3, 1, 4, 4]), 5)

In [9]:
def batch_tokens(tokens: NDArray):
  """batch tokens if not already batched"""
  if len(tokens.shape) == 1:
    return np.array([tokens])
  elif len(tokens.shape) == 2:
    return tokens

  raise ValueError("too many dimensions")

In [10]:
batch = batch_tokens(tokens)
batch

array([[0, 1, 0, 2, 3, 1, 4, 4]])

In [11]:
import torch
import torch.nn.functional as F

In [12]:
def generate_position_embeddings(batch_size, sequence_length: int):
  M = np.eye(sequence_length)
  return torch.tensor(M).repeat(batch_size, 1, 1)

In [13]:
generate_position_embeddings(2, 3)

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]], dtype=torch.float64)

In [14]:
def embed(tokens, num_values = None):
  batch_size, sequence_length = tokens.shape
  num_values = num_values + 2 if num_values is not None else np.max(tokens) + 1 + 2

  one_hot = torch.tensor(np.eye(num_values)[tokens+2])
  positional = generate_position_embeddings(batch_size, sequence_length)


  return torch.cat((one_hot, positional), dim=2)

In [15]:
embeddings = embed(batch, alphabet_size)
embedding_length = embeddings.shape[2]
embeddings

tensor([[[0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]]],
       dtype=torch.float64)

In [16]:
embeddings.shape # should be (batches, sequence length, embedding dimension)

torch.Size([1, 8, 15])

In [17]:
raw_weights = torch.bmm(embeddings, embeddings.transpose(1, 2))
raw_weights

tensor([[[2., 0., 1., 0., 0., 0., 0., 0.],
         [0., 2., 0., 0., 0., 1., 0., 0.],
         [1., 0., 2., 0., 0., 0., 0., 0.],
         [0., 0., 0., 2., 0., 0., 0., 0.],
         [0., 0., 0., 0., 2., 0., 0., 0.],
         [0., 1., 0., 0., 0., 2., 0., 0.],
         [0., 0., 0., 0., 0., 0., 2., 1.],
         [0., 0., 0., 0., 0., 0., 1., 2.]]], dtype=torch.float64)

In [18]:
weights = raw_weights #F.softmax(raw_weights, dim=2)
weights

tensor([[[2., 0., 1., 0., 0., 0., 0., 0.],
         [0., 2., 0., 0., 0., 1., 0., 0.],
         [1., 0., 2., 0., 0., 0., 0., 0.],
         [0., 0., 0., 2., 0., 0., 0., 0.],
         [0., 0., 0., 0., 2., 0., 0., 0.],
         [0., 1., 0., 0., 0., 2., 0., 0.],
         [0., 0., 0., 0., 0., 0., 2., 1.],
         [0., 0., 0., 0., 0., 0., 1., 2.]]], dtype=torch.float64)

In [19]:
y = torch.bmm(weights, embeddings).float()
y[0]

tensor([[0., 0., 3., 0., 0., 0., 0., 2., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 3., 0., 0., 0., 0., 2., 0., 0., 0., 1., 0., 0.],
        [0., 0., 3., 0., 0., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 3., 0., 0., 0., 0., 1., 0., 0., 0., 2., 0., 0.],
        [0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 2., 1.],
        [0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 1., 2.]])

In [20]:
attn_out = y[0]
sequence_length = attn_out.shape[0]
sequence_length, alphabet_size

(8, 5)

In [40]:
helper_tensor = torch.ones((alphabet_size, alphabet_size)) * -1.
helper_tensor.fill_diagonal_(1)
flag_tensor = torch.zeros((alphabet_size, 2))
position_tensor = torch.zeros((alphabet_size, sequence_length))
helper_tensor = torch.cat((flag_tensor, helper_tensor, position_tensor), dim=1)
helper_tensor

tensor([[ 0.,  0.,  1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0., -1.,  1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0., -1., -1.,  1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0., -1., -1., -1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0., -1., -1., -1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.]])

In [32]:
helper_tensor_2 = torch.ones((sequence_length, sequence_length)) * -2
helper_tensor_2.fill_diagonal_(1.)
helper_tensor_2.triu_(0)
zero_tensors = torch.zeros((sequence_length, alphabet_size + 2))
helper_tensor_2 = torch.cat((zero_tensors, helper_tensor_2), dim=1)
helper_tensor_2

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2., -2., -2., -2., -2., -2.,
         -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2., -2., -2., -2., -2.,
         -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2., -2., -2., -2.,
         -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2., -2., -2.,
         -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2., -2.,
         -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,
         -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.]])

In [52]:
# values = attn_out @ helper_tensor.T
values = attn_out @ helper_tensor_2.T

activated = F.relu(values)
activated.sum(dim=1) 

tensor([1., 1., 2., 2., 2., 2., 1., 2.])

In [51]:
weight_tensor = torch.cat((helper_tensor, helper_tensor_2), dim=0)
weight_tensor = torch.cat((torch.eye(embedding_length), weight_tensor))
rand_tensor   = torch.randn((2 * embedding_length + 2, embedding_length))
weight_tensor = torch.cat((weight_tensor, rand_tensor), dim=0)
weight_tensor

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
          0.0000e+00,  0.0000e+00,

In [46]:
weight_tensor_2 = torch.zeros((embedding_length, embedding_length*4))
weight_tensor_2.fill_diagonal_(1)
flag_neuron_1 = torch.zeros((1, embedding_length * 4))
flag_neuron_2 = torch.zeros((1, embedding_length * 4))

flag_neuron_1[0,embedding_length:alphabet_size+embedding_length] = 1
flag_neuron_2[0,alphabet_size+embedding_length:alphabet_size + sequence_length + embedding_length] = 1

weight_tensor_2[0] = flag_neuron_1
weight_tensor_2[1] = flag_neuron_2
weight_tensor_2

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       

In [47]:
xx = torch.nn.Linear(embedding_length, embedding_length * 4, False)
yy = torch.nn.Linear(embedding_length * 4, embedding_length, False)

with torch.no_grad():
  xx.weight = torch.nn.Parameter(weight_tensor)
  yy.weight = torch.nn.Parameter(weight_tensor_2)

In [48]:
input_string

'abacdbee'

In [49]:
yy(torch.nn.ReLU()(xx(y)))

tensor([[[3., 1., 3., 0., 0., 0., 0., 2., 0., 1., 0., 0., 0., 0., 0.],
         [3., 1., 0., 3., 0., 0., 0., 0., 2., 0., 0., 0., 1., 0., 0.],
         [3., 2., 3., 0., 0., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.],
         [2., 2., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
         [2., 2., 0., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
         [3., 2., 0., 3., 0., 0., 0., 0., 1., 0., 0., 0., 2., 0., 0.],
         [3., 1., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 2., 1.],
         [3., 2., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 1., 2.]]],
       grad_fn=<UnsafeViewBackward0>)

In [39]:
y

tensor([[[0., 0., 3., 0., 0., 0., 0., 2., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 3., 0., 0., 0., 0., 2., 0., 0., 0., 1., 0., 0.],
         [0., 0., 3., 0., 0., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
         [0., 0., 0., 3., 0., 0., 0., 0., 1., 0., 0., 0., 2., 0., 0.],
         [0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 2., 1.],
         [0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 1., 2.]]])